## Image classification for quality assurance system
- using VGG16
- modify a classifier 
- freeze all params except for modified classifier
- train and save best model for embedding to Custom App

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import *

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# train data 디렉토리 정의
train_dir = 'path/to/your/train/dataset'

classes = ['cracked','uncracked']

In [ ]:
# 224X224 이미지 처리를 위한 사이즈 조정 및 라벨링

data_transform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                      transforms.ToTensor()])

# torchvision의 datasets.ImageFolder 사용 -> 데이터 로드 및 경로에 따른 자동 라벨링, 지정한 전처리 수행 
train_data = datasets.ImageFolder(train_dir, transform=data_transform) 

In [ ]:
batch_size = 32
num_workers=0

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)

### Load pretrained model & modify the classifier

In [ ]:
vgg16 = models.vgg16(pretrained=True) # VGG16 모델이 미리 학습된 가중치를 사용하여 초기화

# 모든 "feature" 레이어에 대한 훈련 동결
for param in vgg16.features.parameters():
    param.requires_grad = False

n_inputs = vgg16.classifier[6].in_features      # feature 부분의 파라미터 동결 (마지막 출력단 제외)
last_layer = nn.Linear(n_inputs, len(classes))  
vgg16.classifier[6] = last_layer

### Train model  (classifier only)

In [ ]:
import torch.optim as optim

# Set hyperparams
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001) 
n_epochs = 100

In [ ]:
import matplotlib.pyplot as plt


larr = []
best_loss = float('inf')
best_model_wts = None

for epoch in range(1, n_epochs + 1):

    train_loss = 0.0
    num_batches = len(train_loader)

    vgg16.train()  

    for batch_i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  
        output = vgg16(data)  
        
        loss = criterion(output, target)  
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item() 
        print('-', end='')

    print()


    avg_train_loss = train_loss / num_batches
    print(f'Epoch {epoch}, loss: {avg_train_loss:.16f}')
    larr.append(avg_train_loss)

    
    if avg_train_loss < best_loss:
        best_loss = avg_train_loss
        best_model_wts = vgg16.state_dict()

    train_loss = 0.0


# Saving best model
if best_model_wts is not None:
    torch.save(best_model_wts, "best_model_weights.pth")
    print("Best model weights saved to 'best_model_weights.pth'")